In [1]:
import pandas as pd
import numpy as np
import os
from util import HISTORICAL_FILE_JOIN, RESOURCE_DIR 
#pip install mlxtend
from mlxtend.frequent_patterns import apriori, association_rules

 Generates association rules for the results found in the projects history first slice.

In [18]:
#df = pd.read_excel("historical_join.xlsx", keep_default_na=False)
df = pd.read_excel(HISTORICAL_FILE_JOIN, keep_default_na=False)

len(df)
df.head()

,OWNER,PROJECTS,COMMITS,DATE COMMIT,MySQL,PostgreSQL,H2,Oracle,HyperSQL,Redis,...,CouchDB,Impala,Riak KV,Realm,Microsoft Azure Table Storage,Virtuoso-Sql,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,1,2010-07-08 12:23:23+00:00,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Activiti,Activiti,2,2010-07-29 07:38:01+00:00,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Activiti,Activiti,3,2010-08-26 14:46:26+00:00,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Activiti,Activiti,4,2010-09-10 13:23:04+00:00,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Activiti,Activiti,5,2010-09-21 20:47:55+00:00,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
df2 = df.copy()
df2

,OWNER,PROJECTS,COMMITS,DATE COMMIT,MySQL,PostgreSQL,H2,Oracle,HyperSQL,Redis,...,CouchDB,Impala,Riak KV,Realm,Microsoft Azure Table Storage,Virtuoso-Sql,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,1,2010-07-08 12:23:23+00:00,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Activiti,Activiti,2,2010-07-29 07:38:01+00:00,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Activiti,Activiti,3,2010-08-26 14:46:26+00:00,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Activiti,Activiti,4,2010-09-10 13:23:04+00:00,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Activiti,Activiti,5,2010-09-21 20:47:55+00:00,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48741,RoaringBitmap,RoaringBitmap,20,2020-04-25 02:32:30+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48742,RoaringBitmap,RoaringBitmap,21,2021-04-07 15:26:40+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48743,RoaringBitmap,RoaringBitmap,22,2021-11-06 23:14:29+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48744,RoaringBitmap,RoaringBitmap,23,2022-11-14 22:27:49+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Grouping the lines to identify which projects that don't have DB clues

In [20]:
df2 = df2.groupby("PROJECTS").sum()
df2

,COMMITS,MySQL,PostgreSQL,H2,Oracle,HyperSQL,Redis,MongoDB,MS SQL Server,DynamoDB,...,CouchDB,Impala,Riak KV,Realm,Microsoft Azure Table Storage,Virtuoso-Sql,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
PROJECTS,,,,,,,,,,,,,,,,,,,,,
Activiti,8128,81,81,112,75,31,0,0,49,0,...,0,0,0,0,0,0,0,0,0,0
AmazeFileManager,2850,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Android-IMSI-Catcher-Detector,630,0,0,0,0,0,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0
AndroidUtilCode,105,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
AntennaPod,4005,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zipkin,1275,44,10,10,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
zookeeper,1275,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zotfile,66,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Reseting the index column

In [21]:
df2 = df2.reset_index()
df2

,PROJECTS,COMMITS,MySQL,PostgreSQL,H2,Oracle,HyperSQL,Redis,MongoDB,MS SQL Server,...,CouchDB,Impala,Riak KV,Realm,Microsoft Azure Table Storage,Virtuoso-Sql,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,8128,81,81,112,75,31,0,0,49,...,0,0,0,0,0,0,0,0,0,0
1,AmazeFileManager,2850,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Android-IMSI-Catcher-Detector,630,0,0,0,0,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0
3,AndroidUtilCode,105,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AntennaPod,4005,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,zipkin,1275,44,10,10,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
384,zookeeper,1275,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
385,zotfile,66,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
386,zxing,741,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


New dataframe filtering only lines that first version projects

In [22]:
#df10 = df2.mask(df2 > 1, 1)
df10 = df[(df['COMMITS'] == 1)]
df10

,OWNER,PROJECTS,COMMITS,DATE COMMIT,MySQL,PostgreSQL,H2,Oracle,HyperSQL,Redis,...,CouchDB,Impala,Riak KV,Realm,Microsoft Azure Table Storage,Virtuoso-Sql,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,1,2010-07-08 12:23:23+00:00,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
127,airbnb,lottie-android,1,2016-09-24 19:32:20+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146,alibaba,spring-cloud-alibaba,1,2018-10-11 14:14:21+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
181,alibaba,nacos,1,2018-09-18 12:32:55+00:00,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
236,alibaba,fastjson,1,2011-08-13 01:48:45+00:00,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46840,zeromq,jeromq,1,2012-11-20 00:45:59+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46854,zxing,zxing,1,2008-01-22 18:59:47+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46892,apache,camel,1,2007-03-22 18:41:24+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47790,oracle,graal,1,2011-05-05 14:07:00+00:00,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


Removing projects that don't have DB clues the principal dataframe 

In [7]:
#df2 = df2.iloc[ : , 1:]
#df2[(df2 == 0).all(axis=1)].index
df2 = df2.drop(df2[(df2.iloc[ : , 2:] == 0).all(axis=1)].index)
df2

,PROJECTS,COMMITS,MySQL,PostgreSQL,H2,Oracle,HyperSQL,Redis,MongoDB,MS SQL Server,...,CouchDB,Impala,Riak KV,Realm,Microsoft Azure Table Storage,Virtuoso-Sql,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,8128,81,81,112,75,31,0,0,49,...,0,0,0,0,0,0,0,0,0,0
2,Android-IMSI-Catcher-Detector,630,0,0,0,0,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0
3,AndroidUtilCode,105,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,AnySoftKeyboard,3916,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,AsciidocFX,210,2,2,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,yacy_search_server,10585,136,136,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
381,zaproxy,5778,67,0,0,55,107,0,0,0,...,0,0,0,0,0,0,0,0,0,0
382,zeppelin,2628,53,56,48,0,0,53,31,5,...,0,0,0,0,0,0,0,47,0,0
383,zipkin,1275,44,10,10,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


Removing projects that don't have DB clues in the first version 

In [23]:
df3 = df[(df['COMMITS'] == 1)]
#df3.drop(["PROJECTS", "COMMITS"], axis=1, inplace=True)
df3 = df3.drop(df3[(df3.iloc[:,2:] == 0).all(axis=1)].index) # deletando as linhas que possuem valor 0 em todas as colunas
df3

,OWNER,PROJECTS,COMMITS,DATE COMMIT,MySQL,PostgreSQL,H2,Oracle,HyperSQL,Redis,...,CouchDB,Impala,Riak KV,Realm,Microsoft Azure Table Storage,Virtuoso-Sql,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,1,2010-07-08 12:23:23+00:00,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
127,airbnb,lottie-android,1,2016-09-24 19:32:20+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146,alibaba,spring-cloud-alibaba,1,2018-10-11 14:14:21+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
181,alibaba,nacos,1,2018-09-18 12:32:55+00:00,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
236,alibaba,fastjson,1,2011-08-13 01:48:45+00:00,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46840,zeromq,jeromq,1,2012-11-20 00:45:59+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46854,zxing,zxing,1,2008-01-22 18:59:47+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46892,apache,camel,1,2007-03-22 18:41:24+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47790,oracle,graal,1,2011-05-05 14:07:00+00:00,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


Merging dataframe grouping and dataframe only first version with DB clues

In [24]:
#df3.merge(df4, on=['PROJECTS'], how='outer', suffixes=['', '_'], indicator=True)
df_mer= pd.merge(df2, df3, on= 'PROJECTS', how='outer', suffixes=['', '_'], indicator=True)
df_mer

,PROJECTS,COMMITS,MySQL,PostgreSQL,H2,Oracle,HyperSQL,Redis,MongoDB,MS SQL Server,...,Impala_,Riak KV_,Realm_,Microsoft Azure Table Storage_,Virtuoso-Sql_,Join_Ignite-Sql_,Join_Ignite-NoSql_,Join_Virtuoso-Sql_,Join_Virtuoso-NoSql_,_merge
0,Activiti,8128,81,81,112,75,31,0,0,49,...,0,0,0,0,0,0,0,0,0,both
1,AmazeFileManager,2850,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
2,Android-IMSI-Catcher-Detector,630,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
3,AndroidUtilCode,105,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,both
4,AntennaPod,4005,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,zipkin,1275,44,10,10,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,both
386,zookeeper,1275,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
387,zotfile,66,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
388,zxing,741,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both


Filtering diference in the first version

In [25]:
df5 = df_mer[(df_mer['_merge'].str.startswith('left'))]
df5

,PROJECTS,COMMITS,MySQL,PostgreSQL,H2,Oracle,HyperSQL,Redis,MongoDB,MS SQL Server,...,Impala_,Riak KV_,Realm_,Microsoft Azure Table Storage_,Virtuoso-Sql_,Join_Ignite-Sql_,Join_Ignite-NoSql_,Join_Virtuoso-Sql_,Join_Virtuoso-NoSql_,_merge


Merging dataframe only lines that first version projects and dataframe only lines in the first version and not DB clues

In [26]:
df_mer2= pd.merge(df10, df5.iloc[:,:1], on= 'PROJECTS', how='outer', suffixes=['', '_'], indicator=True)
df_mer2

,OWNER,PROJECTS,COMMITS,DATE COMMIT,MySQL,PostgreSQL,H2,Oracle,HyperSQL,Redis,...,Impala,Riak KV,Realm,Microsoft Azure Table Storage,Virtuoso-Sql,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql,_merge
0,Activiti,Activiti,1,2010-07-08 12:23:23+00:00,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,left_only
1,airbnb,lottie-android,1,2016-09-24 19:32:20+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,left_only
2,alibaba,spring-cloud-alibaba,1,2018-10-11 14:14:21+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,left_only
3,alibaba,nacos,1,2018-09-18 12:32:55+00:00,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,left_only
4,alibaba,fastjson,1,2011-08-13 01:48:45+00:00,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,zeromq,jeromq,1,2012-11-20 00:45:59+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,left_only
386,zxing,zxing,1,2008-01-22 18:59:47+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,left_only
387,apache,camel,1,2007-03-22 18:41:24+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,left_only
388,oracle,graal,1,2011-05-05 14:07:00+00:00,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,left_only


Filtering only lines in the first version and not DB clues

In [27]:
df6 = df_mer2[(df_mer2['_merge'].str.startswith('bo'))]
df6

,OWNER,PROJECTS,COMMITS,DATE COMMIT,MySQL,PostgreSQL,H2,Oracle,HyperSQL,Redis,...,Impala,Riak KV,Realm,Microsoft Azure Table Storage,Virtuoso-Sql,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql,_merge


Concatening dataframes

In [28]:
df_concat2 =  pd.concat([df3, df6]).drop_duplicates()
df_concat2

,OWNER,PROJECTS,COMMITS,DATE COMMIT,MySQL,PostgreSQL,H2,Oracle,HyperSQL,Redis,...,Impala,Riak KV,Realm,Microsoft Azure Table Storage,Virtuoso-Sql,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql,_merge
0,Activiti,Activiti,1,2010-07-08 12:23:23+00:00,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,NaN
127,airbnb,lottie-android,1,2016-09-24 19:32:20+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
146,alibaba,spring-cloud-alibaba,1,2018-10-11 14:14:21+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
181,alibaba,nacos,1,2018-09-18 12:32:55+00:00,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
236,alibaba,fastjson,1,2011-08-13 01:48:45+00:00,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46840,zeromq,jeromq,1,2012-11-20 00:45:59+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
46854,zxing,zxing,1,2008-01-22 18:59:47+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
46892,apache,camel,1,2007-03-22 18:41:24+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
47790,oracle,graal,1,2011-05-05 14:07:00+00:00,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [14]:
#df2 = df2.astype(np.uint8)

In [29]:
df_concat2.drop(["PROJECTS","COMMITS","_merge"], axis=1, inplace=True)
df_concat2

,OWNER,DATE COMMIT,MySQL,PostgreSQL,H2,Oracle,HyperSQL,Redis,MongoDB,MS SQL Server,...,CouchDB,Impala,Riak KV,Realm,Microsoft Azure Table Storage,Virtuoso-Sql,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,2010-07-08 12:23:23+00:00,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
127,airbnb,2016-09-24 19:32:20+00:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146,alibaba,2018-10-11 14:14:21+00:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
181,alibaba,2018-09-18 12:32:55+00:00,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
236,alibaba,2011-08-13 01:48:45+00:00,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46840,zeromq,2012-11-20 00:45:59+00:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46854,zxing,2008-01-22 18:59:47+00:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46892,apache,2007-03-22 18:41:24+00:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47790,oracle,2011-05-05 14:07:00+00:00,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


ASSOCIATION RULES

In [30]:
frq_items = apriori(df_concat2, min_support = 0.021, use_colnames = True)

ValueError: The allowed values for a DataFrame are True, False, 0, 1. Found value Activiti

In [17]:
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)

In [18]:
# Sorting the rules by the lift measure 
rules = rules.sort_values('lift', ascending =False)
print(rules.head())

                                            antecedents  \
9088  (Informix, HyperSQL, MySQL_Maria DB, PostgreSQ...   
7779                         (Informix, MySQL_Maria DB)   
7764        (Informix, PostgreSQL_ CockroachDB, Oracle)   
9583                   (H2, Informix, HyperSQL, Oracle)   
9586    (H2, Informix, PostgreSQL_ CockroachDB, Oracle)   

                                            consequents  antecedent support  \
9088                              (H2, SQLite, IBM DB2)            0.025641   
7779  (SQLite, PostgreSQL_ CockroachDB, Oracle, IBM ...            0.025641   
7764                  (MySQL_Maria DB, SQLite, IBM DB2)            0.025641   
9583         (PostgreSQL_ CockroachDB, SQLite, IBM DB2)            0.025641   
9586                        (HyperSQL, SQLite, IBM DB2)            0.025641   

      consequent support   support  confidence  lift  leverage  conviction  
9088            0.025641  0.025641         1.0  39.0  0.024984         inf  
7779            

In [19]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
9088,"(Informix, HyperSQL, MySQL_Maria DB, PostgreSQ...","(H2, SQLite, IBM DB2)",0.025641,0.025641,0.025641,1.000000,39.000000,0.024984,inf
7779,"(Informix, MySQL_Maria DB)","(SQLite, PostgreSQL_ CockroachDB, Oracle, IBM ...",0.025641,0.025641,0.025641,1.000000,39.000000,0.024984,inf
7764,"(Informix, PostgreSQL_ CockroachDB, Oracle)","(MySQL_Maria DB, SQLite, IBM DB2)",0.025641,0.025641,0.025641,1.000000,39.000000,0.024984,inf
9583,"(H2, Informix, HyperSQL, Oracle)","(PostgreSQL_ CockroachDB, SQLite, IBM DB2)",0.025641,0.025641,0.025641,1.000000,39.000000,0.024984,inf
9586,"(H2, Informix, PostgreSQL_ CockroachDB, Oracle)","(HyperSQL, SQLite, IBM DB2)",0.025641,0.025641,0.025641,1.000000,39.000000,0.024984,inf
...,...,...,...,...,...,...,...,...,...
18,(MySQL_Maria DB),(SQLite),0.333333,0.087179,0.061538,0.184615,2.117647,0.032479,1.119497
1,(MySQL_Maria DB),(HyperSQL),0.333333,0.138462,0.092308,0.276923,2.000000,0.046154,1.191489
0,(HyperSQL),(MySQL_Maria DB),0.138462,0.333333,0.092308,0.666667,2.000000,0.046154,2.000000
5,(MySQL_Maria DB),(Redis),0.333333,0.066667,0.030769,0.092308,1.384615,0.008547,1.028249


In [20]:
rules['antecedent_len'] = rules['antecedents'].apply(lambda x: len(x))
rules['consequente_len'] = rules['consequents'].apply(lambda x: len(x))

Filtering only the rules of size 2, that is, 1 antecedent and 1 consequent. 

In [21]:
rules_filter = rules[(rules['antecedent_len'] == 1) & (rules['consequente_len'] == 1)]

Showing all filtered rules.

In [22]:
rules_filter

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequente_len
107,(SAP Adaptive Server),(SAP SQL Anywhere),0.046154,0.046154,0.046154,1.000000,21.666667,0.044024,inf,1,1
106,(SAP SQL Anywhere),(SAP Adaptive Server),0.046154,0.046154,0.046154,1.000000,21.666667,0.044024,inf,1,1
89,(IBM DB2),(Informix),0.056410,0.030769,0.025641,0.454545,14.772727,0.023905,1.776923,1,1
88,(Informix),(IBM DB2),0.030769,0.056410,0.025641,0.833333,14.772727,0.023905,5.661538,1,1
99,(IBM DB2),(SAP Adaptive Server),0.056410,0.046154,0.030769,0.545455,11.818182,0.028166,2.098462,1,1
...,...,...,...,...,...,...,...,...,...,...,...
18,(MySQL_Maria DB),(SQLite),0.333333,0.087179,0.061538,0.184615,2.117647,0.032479,1.119497,1,1
1,(MySQL_Maria DB),(HyperSQL),0.333333,0.138462,0.092308,0.276923,2.000000,0.046154,1.191489,1,1
0,(HyperSQL),(MySQL_Maria DB),0.138462,0.333333,0.092308,0.666667,2.000000,0.046154,2.000000,1,1
5,(MySQL_Maria DB),(Redis),0.333333,0.066667,0.030769,0.092308,1.384615,0.008547,1.028249,1,1


In [23]:
HISTORICAL_RULES_FILE = RESOURCE_DIR + os.sep + 'historical_rulesv1.xlsx'  #'seq_patterns/testes/historical_rules_commit1_197reg_021.xlsx'
rules_filter = rules_filter.to_excel(HISTORICAL_RULES_FILE, index=False)